In [1]:
from datetime import datetime
from pathlib import Path
import gsw
import numpy as np
import pandas as pd
# ignore pandas "educational" performance warnings
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

import xarray as xr
import pyarrow.parquet as pq
import requests as rq
import os
from urllib.parse import urljoin, urlencode
import argo_tools as at
import argopy
from argopy import DataFetcher as ArgoDataFetcher
gdac_path = '/vortexfs1/share/boom/data/nc2pqt_test/'
outdir_nc = '/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/'
# gdac_path = '/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/'
# outdir_nc = '/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/GDAC_test/'

outdir_pqt = '/vortexfs1/share/boom/data/nc2pqt_test/pqt2/'
# argopy.set_options(mode='expert',src='gdac',ftp=local_gdac)
from pprint import pprint
import warnings
import glob
import psutil
import time
import multiprocessing
import gc
import json

ModuleNotFoundError: No module named 'gsw'

#### Downloading Argo profiles

If you already have the profiles stored somewhere, you can skip this. Otherwise, if you want to download them, uncomment the next cell, and pass to `at.argo_dac` the appropriate arguments to download the files you desire (see `argo_tools.py` for details about the arguments).

In [2]:
wmos, df2, wmo_fp = at.argo_gdac(gdac_path=gdac_path,save_to=outdir_nc, download_individual_profs=False, skip_downloads=False, dryrun=True, checktime=True)

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/argo_tools.py:77: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(gdac_file,delimiter=',',header=8,parse_dates=['date','date_update'],


#### Importing iterators from [`itertools`](https://docs.python.org/3/library/itertools.html)
* `islice` returns selected elements from iterable
* `batched` split the iterable object into tuples of prescribed length _n_ (if `length(iterable)%n~=0`, the last tuple is shorter than _n_).

In [3]:
import sys
import itertools
from itertools import islice

if sys.version_info >= (3, 12):
    from itertools import batched
else:
    try:
        from more_itertools import batched
    except ImportError:
        def batched(iterable, chunk_size):
            iterator = iter(iterable)
            while chunk := tuple(islice(iterator, chunk_size)):
                yield chunk

In [4]:
VARS = ['LATITUDE','LONGITUDE','JULD','CYCLE_NUMBER','PLATFORM_NUMBER',
 'PRES',
 'PRES_QC',
 'PRES_ADJUSTED',
 'PRES_ADJUSTED_QC',
 'PRES_ADJUSTED_ERROR',
 'TEMP',
 'TEMP_QC',
 'TEMP_dPRES',
 'TEMP_ADJUSTED',
 'TEMP_ADJUSTED_QC',
 'TEMP_ADJUSTED_ERROR',
 'PSAL',
 'PSAL_QC',
 'PSAL_dPRES',
 'PSAL_ADJUSTED',
 'PSAL_ADJUSTED_QC',
 'PSAL_ADJUSTED_ERROR',
 'DOXY',
 'DOXY_QC',
 'DOXY_dPRES',
 'DOXY_ADJUSTED',
 'DOXY_ADJUSTED_QC',
 'DOXY_ADJUSTED_ERROR',
 'BBP',
 'BBP_QC',
 'BBP_dPRES',
 'BBP_ADJUSTED',
 'BBP_ADJUSTED_QC',
 'BBP_ADJUSTED_ERROR',
 'BBP470',
 'BBP470_QC',
 'BBP470_dPRES',
 'BBP470_ADJUSTED',
 'BBP470_ADJUSTED_QC',
 'BBP470_ADJUSTED_ERROR',
 'BBP532',
 'BBP532_QC',
 'BBP532_dPRES',
 'BBP532_ADJUSTED',
 'BBP532_ADJUSTED_QC',
 'BBP532_ADJUSTED_ERROR',
 'BBP700',
 'BBP700_QC',
 'BBP700_dPRES',
 'BBP700_ADJUSTED',
 'BBP700_ADJUSTED_QC',
 'BBP700_ADJUSTED_ERROR',
 'TURBIDITY',
 'TURBIDITY_QC',
 'TURBIDITY_dPRES',
 'TURBIDITY_ADJUSTED',
 'TURBIDITY_ADJUSTED_QC',
 'TURBIDITY_ADJUSTED_ERROR',
 'CP',
 'CP_QC',
 'CP_dPRES',
 'CP_ADJUSTED',
 'CP_ADJUSTED_QC',
 'CP_ADJUSTED_ERROR',
 'CP660',
 'CP660_QC',
 'CP660_dPRES',
 'CP660_ADJUSTED',
 'CP660_ADJUSTED_QC',
 'CP660_ADJUSTED_ERROR',
 'CHLA',
 'CHLA_QC',
 'CHLA_dPRES',
 'CHLA_ADJUSTED',
 'CHLA_ADJUSTED_QC',
 'CHLA_ADJUSTED_ERROR',
 'CDOM',
 'CDOM_QC',
 'CDOM_dPRES',
 'CDOM_ADJUSTED',
 'CDOM_ADJUSTED_QC',
 'CDOM_ADJUSTED_ERROR',
 'NITRATE',
 'NITRATE_QC',
 'NITRATE_dPRES',
 'NITRATE_ADJUSTED',
 'NITRATE_ADJUSTED_QC',
 'NITRATE_ADJUSTED_ERROR',
 'BISULFIDE',
 'BISULFIDE_QC',
 'BISULFIDE_dPRES',
 'BISULFIDE_ADJUSTED',
 'BISULFIDE_ADJUSTED_QC',
 'BISULFIDE_ADJUSTED_ERROR',
 'PH_IN_SITU_TOTAL',
 'PH_IN_SITU_TOTAL_QC',
 'PH_IN_SITU_TOTAL_dPRES',
 'PH_IN_SITU_TOTAL_ADJUSTED',
 'PH_IN_SITU_TOTAL_ADJUSTED_QC',
 'PH_IN_SITU_TOTAL_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE',
 'DOWN_IRRADIANCE_QC',
 'DOWN_IRRADIANCE_dPRES',
 'DOWN_IRRADIANCE_ADJUSTED',
 'DOWN_IRRADIANCE_ADJUSTED_QC',
 'DOWN_IRRADIANCE_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE380',
 'DOWN_IRRADIANCE380_QC',
 'DOWN_IRRADIANCE380_dPRES',
 'DOWN_IRRADIANCE380_ADJUSTED',
 'DOWN_IRRADIANCE380_ADJUSTED_QC',
 'DOWN_IRRADIANCE380_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE412',
 'DOWN_IRRADIANCE412_QC',
 'DOWN_IRRADIANCE412_dPRES',
 'DOWN_IRRADIANCE412_ADJUSTED',
 'DOWN_IRRADIANCE412_ADJUSTED_QC',
 'DOWN_IRRADIANCE412_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE443',
 'DOWN_IRRADIANCE443_QC',
 'DOWN_IRRADIANCE443_dPRES',
 'DOWN_IRRADIANCE443_ADJUSTED',
 'DOWN_IRRADIANCE443_ADJUSTED_QC',
 'DOWN_IRRADIANCE443_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE490',
 'DOWN_IRRADIANCE490_QC',
 'DOWN_IRRADIANCE490_dPRES',
 'DOWN_IRRADIANCE490_ADJUSTED',
 'DOWN_IRRADIANCE490_ADJUSTED_QC',
 'DOWN_IRRADIANCE490_ADJUSTED_ERROR',
 'DOWN_IRRADIANCE555',
 'DOWN_IRRADIANCE555_QC',
 'DOWN_IRRADIANCE555_dPRES',
 'DOWN_IRRADIANCE555_ADJUSTED',
 'DOWN_IRRADIANCE555_ADJUSTED_QC',
 'DOWN_IRRADIANCE555_ADJUSTED_ERROR',
 'UP_IRRADIANCE',
 'UP_IRRADIANCE_QC',
 'UP_IRRADIANCE_dPRES',
 'UP_IRRADIANCE_ADJUSTED',
 'UP_IRRADIANCE_ADJUSTED_QC',
 'UP_IRRADIANCE_ADJUSTED_ERROR',
 'UP_IRRADIANCE380',
 'UP_IRRADIANCE380_QC',
 'UP_IRRADIANCE380_dPRES',
 'UP_IRRADIANCE380_ADJUSTED',
 'UP_IRRADIANCE380_ADJUSTED_QC',
 'UP_IRRADIANCE380_ADJUSTED_ERROR',
 'UP_IRRADIANCE412',
 'UP_IRRADIANCE412_QC',
 'UP_IRRADIANCE412_dPRES',
 'UP_IRRADIANCE412_ADJUSTED',
 'UP_IRRADIANCE412_ADJUSTED_QC',
 'UP_IRRADIANCE412_ADJUSTED_ERROR',
 'UP_IRRADIANCE443',
 'UP_IRRADIANCE443_QC',
 'UP_IRRADIANCE443_dPRES',
 'UP_IRRADIANCE443_ADJUSTED',
 'UP_IRRADIANCE443_ADJUSTED_QC',
 'UP_IRRADIANCE443_ADJUSTED_ERROR',
 'UP_IRRADIANCE490',
 'UP_IRRADIANCE490_QC',
 'UP_IRRADIANCE490_dPRES',
 'UP_IRRADIANCE490_ADJUSTED',
 'UP_IRRADIANCE490_ADJUSTED_QC',
 'UP_IRRADIANCE490_ADJUSTED_ERROR',
 'UP_IRRADIANCE555',
 'UP_IRRADIANCE555_QC',
 'UP_IRRADIANCE555_dPRES',
 'UP_IRRADIANCE555_ADJUSTED',
 'UP_IRRADIANCE555_ADJUSTED_QC',
 'UP_IRRADIANCE555_ADJUSTED_ERROR',
 'DOWNWELLING_PAR',
 'DOWNWELLING_PAR_QC',
 'DOWNWELLING_PAR_dPRES',
 'DOWNWELLING_PAR_ADJUSTED',
 'DOWNWELLING_PAR_ADJUSTED_QC',
 'DOWNWELLING_PAR_ADJUSTED_ERROR']

VARS = sorted(VARS)

### File conversion

The conversion from `nc` to `parquet` is parallelized. All you need to do is comment/uncomment the appropriate line in the next cell, selecting if you want to process the files that you just downloaded (first line) or other files (second line, modify to include your desired path).
The new parquet files will be stored in the directory `outdir_pqt` that you specified earlier.

The default is for the code to execute over multiple processes. If you want to execute it on a single thread, set `single_process = True`

In [ ]:
%%time

single_process = False
if not single_process:
    import multiprocessing

def xr2pqt(rank,files_list,loop_id):
    df_list = []
    df_memory = 0
    counter = 0
    rank_str = "#" + str(rank) + ": "
    nb_files = len(files_list)
    argo_file_fail = []
    for argo_file in files_list:
        counter += 1
        if counter%10==0:
            print(rank_str + "processing file " + str(counter) + " of " + str(nb_files))
            
        try:
            ds = xr.load_dataset(argo_file, engine="argo") #loading into memory the profile
        except:
            print(rank_str + 'Failed on ' + str(argo_file))
            argo_file_fail.append(argo_file)
            continue

        # some floats don't have all the vars specified in VARS
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        # df = ds.to_dataframe()

        if not df.empty:
            # for col in VARS:
            #     if col not in invars:
            #         df[col] = np.nan #ensures that all parquet files have all the VARS as columns
            # df_memory += df.memory_usage().sum()/(10**6) # tracking memory usage (in MB)
            df_list.append(df)

            df = None
            ds = None
            del df
            del ds
            gc.collect()

    # store to parquet once a large enough dataframe has been created
    print(rank_str + "Storing to parquet...")

    try:
        df_list = pd.concat(df_list, axis=0) # it automatically adds NaNs where needed
    except:
        print(rank_str + 'Could not concatenate pandas dataframes')
        print(rank_str + 'Failed on ' + str(argo_file) + '. Caution: more files might be affected.')
        print(rank_str + 'Data frames list:')
        print(df_list)
        return argo_file_fail    

    df_memory = df_list.memory_usage().sum()/(1024**2)
    if df_memory < 1e3:
        print(rank_str + "In-memory filesize: " + "{:.2f}".format(df_memory) + " MB")
    else:
        print(rank_str + "In-memory filesize: " + "{:.2f}".format(df_memory/1024) + " GB")

    parquet_filename = outdir_pqt + "test_profiles_levels_" + str(rank) + "_" + str(loop_id) + ".parquet"
    df_list.to_parquet(parquet_filename)
    print(rank_str + str(parquet_filename) + " stored.")

    df_list = None
    del df_list
    gc.collect()

    return argo_file_fail
    
############################################################################################################

def poolParams(flist,nc_size_per_pqt):
    size_flist = []
    for f in flist:
        try:
            f_size = os.path.getsize(f)/1024**2
            size_flist.append( f_size ) #size in MB
        except:
            if not os.path.isfile(f):
                gdac_root = 'https://usgodae.org/pub/outgoing/argo/dac/'
                fpath = os.path.join( *f.split(os.path.sep)[-3:] )
                response = at.get_func( gdac_root + fpath )
                if response.status_code == 404:
                    print('File ' + f + ' returned 404 error from URL ' + str(gdac_root+fpath) + '. Skipping it.')
                else:
                    print('File ' + f + ' likely present at URL ' + str(gdac_root+fpath) + '. You might want to check why it is not in the local drive.')
            else:
                print('File ' + f + ' seems to exist in the local drive: not sure what is going on here.')
            continue
            
    size_tot = sum(size_flist)
    NPROC = int(np.ceil(size_tot/nc_size_per_pqt))
    size_per_proc = size_tot/NPROC

    print('')
    print('Size per processor (MB)')
    print(size_per_proc)
    print('')
    
    ids_sort = np.argsort(np.array(size_flist))
    
    chunks_ids = []
    x = np.copy(ids_sort)
    
    for j in range(NPROC):
        chunk_ids = []
        chunk_size = 0
        while ((chunk_size<size_per_proc) and (len(x) > 0)):
            if len(chunk_ids)%2 == 0:
                chunk_ids.append(x[-1])
                x = x[:-1]
            else:
                chunk_ids.append(x[0])
                x = x[1:]
            chunk_size = sum(np.asarray(size_flist)[chunk_ids])
        print(chunk_size)
        chunks_ids.append(chunk_ids)
    
    if len(x) > 0:
        warnings.warn(str(len(x)) + " files have not been assigned to a processor.")
    
    print('')
    chunks=[]
    skip_proc = 0
    total_memory = 0
    for j,chunk_ids in enumerate(chunks_ids):
        print('Size in processor ' + str(j) + ' (MB):')
        size_proc = sum(np.asarray(size_flist)[chunk_ids])
        total_memory += size_proc
        print(size_proc)
        if size_proc == 0:
            skip_proc += 1
            continue
        chunk = [flist[k] for k in chunk_ids]
        chunks.append(chunk)

    NPROC -= skip_proc
        
    print('')
    print("Using " + str(NPROC) + " processors")
    
    return NPROC, chunks, size_per_proc

########################################################

def inMemorySize(flist):
    mem = []
    print('Parsing in-memory usage of Argo files.')
    print('This might take a while, and if you have an old file with this information, passing its path could be faster.')
    print('You can also decide to use in-disk memory to optimize file size conversion by calling def poolParams() instead, although it performs worse.')
    for file in flist:
        try:
            ds = xr.open_dataset(file, engine="argo")
            mem.append( ds.nbytes/(1024*1024) )
        except:
            print('skipping ' + file)
            mem.append(-1)
            
    with open( outdir_pqt + 'inmemory_file_size.json', 'w') as f:
        json.dump(mem, f)

    print('done.')
    
    return mem

def poolParamsMem(flist,inmem_size_per_pqt,fmem_path=None):

    if fmem_path is None:
        fmem = inMemorySize(flist)
    else:    
        with open(fmem_path, 'r') as f:
            fmem = json.load(f)
            
    size_tot = sum(fmem)
    NPROC = int(np.ceil(size_tot/inmem_size_per_pqt))
    size_per_proc = size_tot/NPROC

    print('')
    print('Size per processor (MB)')
    print(size_per_proc)
    print('')
    
    ids_sort = np.argsort(np.array(fmem))
    
    chunks_ids = []
    x = np.copy(ids_sort)
    
    for j in range(NPROC):
        chunk_ids = []
        chunk_size = 0
        while ((chunk_size<size_per_proc) and (len(x) > 0)):
            if len(chunk_ids)%2 == 0:
                chunk_ids.append(x[-1])
                x = x[:-1]
            else:
                chunk_ids.append(x[0])
                x = x[1:]
            chunk_size = sum(np.asarray(fmem)[chunk_ids])
        print(chunk_size)
        chunks_ids.append(chunk_ids)
    
    if len(x) > 0:
        warnings.warn(str(len(x)) + " files have not been assigned to a processor.")
    
    print('')
    chunks=[]
    skip_proc = 0
    total_memory = 0
    for j,chunk_ids in enumerate(chunks_ids):
        size_proc = sum(np.asarray(fmem)[chunk_ids])
        total_memory += size_proc
        print('In-memory size in processor ' + str(j) + ' (MB): ' + str(size_proc) )
        # print(size_proc)
        if size_proc == 0:
            skip_proc += 1
            continue
        chunk = [flist[k] for k in chunk_ids]
        chunks.append(chunk)

    NPROC -= skip_proc
        
    print('')
    print("Using " + str(NPROC) + " processors")
    
    return NPROC, chunks, size_per_proc

############################################################################################################

# Metadata
flist = wmo_fp
metadata_dir = outdir_pqt + "metadata/"
Path(metadata_dir).mkdir(parents= True, exist_ok= True)
parquet_filename = metadata_dir + "test_metadata.parquet"
df2.to_parquet(parquet_filename)
print(str(parquet_filename) + " stored.")

# Profiles
print("Processing " + str(len(flist)) + " files.")

if not single_process:
    nc_size_per_pqt = 40 # Empirically, 40 MB of average .nc file size gives in-memory sizes between 100-330 MB, which is what Dask recommens
    NPROC, chunks,size_per_proc = poolParams(flist,nc_size_per_pqt)

# if not single_process:
#     inmem_size_per_pqt = 300
#     NPROC, chunks,size_per_proc = poolParamsMem(flist,inmem_size_per_pqt)

# fixing max nb of processes to prevent bottleneck likely due to I/O on disk queing operations and filling up the memory
MAXPROC = 100
if size_per_proc > 300:
    MAXPROC = 20

if NPROC > MAXPROC and not single_process:
    print("Estimated number of processors might create bottleneck issues. Forcing to use " + str(MAXPROC) + " processors at a time.")
    # force to use at most MAXPROC processes, by looping over chunks
    full_loops = NPROC//MAXPROC  #nb of loops to use at most MAXPROC
    RESPROC = NPROC%MAXPROC   #nb of residual processors after the loops
    
    i_start = 0
    i_end   = 0
    failed_files = []
    for full_loop in range(full_loops):
        i_start = MAXPROC*full_loop
        i_end   = MAXPROC*(full_loop+1)
        pool_obj = multiprocessing.Pool(processes=MAXPROC)
        failed_files.append( pool_obj.starmap(xr2pqt, [(rank, chunk, full_loop) for rank, chunk in enumerate(chunks[i_start:i_end])] ) )
        pool_obj.close()

    # multiprocessing across residual processor pool with NPROC<MAXPROC
    if RESPROC > 0:
        pool_obj = multiprocessing.Pool(processes=RESPROC)
        failed_files.append( pool_obj.starmap(xr2pqt, [(rank, chunk, full_loop+1) for rank, chunk in enumerate(chunks[(i_end+1):])] ) )
        pool_obj.close()

elif NPROC > 1 and not single_process:
    failed_files = []
    pool_obj = multiprocessing.Pool(processes=NPROC)
    failed_files.append( pool_obj.starmap(xr2pqt, [(rank, chunk, 0) for rank, chunk in enumerate(chunks)] ) )
    pool_obj.close()

else:
    failed_files = xr2pqt(0,flist,0)

failed = []
for f in failed_files:
    for g in f:
        if len(g) > 0:
            for h in g:
                failed.append(h)
                print(h)

print('Files that encountered an error and were not converted:')
print(failed)

/vortexfs1/share/boom/data/nc2pqt_test/pqt2/metadata/test_metadata.parquet stored.
Processing 2239 files.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902104/3902104_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/3902104/3902104_Sprof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902115/3902115_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/3902115/3902115_Sprof.nc. Skipping it.


...

File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/7900586/7900586_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/7900586/7900586_Sprof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/7900587/7900587_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/7900587/7900587_Sprof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/csio/2902836/2902836_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/csio/2902836/2902836_Sprof.nc. Skipping it.

Size per processor (MB)
39.96117754656859

99.08544540405273
97.73884105682373
96.52665042877197
89.80551052093506
87.98799419403076
87.86990928649902
87.43379592895508
85.42619132995605
83.34097290039062
82.52131366729736
81.72994232177734
81.44808578491211
81.2982702255249
80.78289413452148
79.62730979919434
76.59406471252441
72.35435104370117
69.35292625427246
65.87495422363281
65.13488674163818
63.51381015777588
61.91689109802246
57.574278831481934
57.280272483825684
56.58053207397461
54.36483383178711
52.90372276306152
52.17085838317871
51.750932693481445
50.643943786621094
50.5104455947876
48.93765640258789
48.876712799072266
48.60991191864014
48.206050872802734
47.92483425140381
47.206130027770996
46.32355880737305
45.93830585479736
44.92294502258301
44.75474262237549
44.4288272857666

#### Converting all metadata to parquet

In [ ]:
parquet_filename = outdir_pqt + "test_metadata.parquet"
df_list.to_parquet(parquet_filename)
print(str(parquet_filename) + " stored.")

In [ ]:
def metadata2pqt(rank,files_list,refVARS):
    rank_str = "#" + str(rank) + ": "
    
    df_list = []
    df_memory = 0
    counter = 0
    nb_files = len(files_list)
    argo_file_fail = []
    for argo_file in files_list:
        counter += 1
        if counter%10==0:
            print(rank_str + "processing file " + str(counter) + " of " + str(nb_files))
            
        try:
            ds = xr.load_dataset(argo_file, engine="argo") #loading into memory the profile
        except:
            print(rank_str + 'Failed on ' + str(argo_file))
            argo_file_fail.append(argo_file)
        
        d0 = ds[refVARS]

        # adding dimension = PLATFORM_NUMBER and its value to the current array
        d0=d0.assign_coords(PLATFORM_NUMBER=ds.PLATFORM_NUMBER.values[0])
        for da in d0.data_vars:
            d0[da]=d0[da].expand_dims(dim={"PLATFORM_NUMBER": 1}, axis=0)
        
        df = d0.to_dataframe()
        df_memory += df.memory_usage().sum()/(10**6) # tracking memory usage (in GB)
        df_list.append(df)

        if df_memory > 200:
            print(rank_str + "In-memory filesize: " + "{:.2f}".format(df_memory) + " MB. This is above the recommended size for parquet.")
    
    df_list = pd.concat(df_list)

    print(rank_str + "Returning list of dataframes to main processor.")

    return df_list, argo_file_fail

####################################

flist = glob.glob("/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/*/*/*_Sprof.nc")
# refVARS = ['PARAMETER', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 'SCIENTIFIC_CALIB_DATE']
refVARS = ["DATA_TYPE","FORMAT_VERSION","HANDBOOK_VERSION","REFERENCE_DATE_TIME","DATE_CREATION","DATE_UPDATE"]
nb_of_checks = len(flist)

NPROC = 100
if NPROC > nb_of_checks:
    NPROC = nb_of_checks
CHUNK_SZ = int(np.ceil(nb_of_checks/NPROC))
chunks = batched(flist,CHUNK_SZ)

print(CHUNK_SZ)

pool_obj = multiprocessing.Pool(processes=NPROC)
outputs = pool_obj.starmap(metadata2pqt, [(rank, chunk, refVARS) for rank, chunk in enumerate(chunks)])
pool_obj.close()

df_list = []
failed_files = []
for processor_output in outputs:
    df_list.append(processor_output[0])
    if len(processor_output[1])>0:
        for f in processor_output[1]:
            failed_files.append(f)
df_list = pd.concat(df_list)

parquet_filename = outdir_pqt + "test_metadata.parquet"
df_list.to_parquet(parquet_filename)
print(str(parquet_filename) + " stored.")

print("Failed files:")
print(failed_files)

#### Reading from parquet

There are a couple of way to read parquet files. One is directly using pandas (make sure you have pyarrow, fastparquet or other suitable engine installed), the other is with Dask. Generally speaking, you'll want to use Dask if you need a large amount of data at the same time so that you can benefit from its parallelization. You should avoid Dask and just go for pandas whenever the data fits in your RAM.

When reading parquet files with pandas, you can either specificy the file name if you know which file you want, or the directory containing all the parquet files. In latter case if you apply any filter, pandas and pyarrow will sort through all the files in the folder, reading into memory only the subsets that satisfy your filter.

In [7]:
sel = [("PLATFORM_NUMBER", "==", 6901494)]
df = pd.read_parquet( glob.glob(outdir_pqt + "test_profiles*") , engine='pyarrow', filters = sel )
sorted(df.columns.to_list())

['BBP700',
 'BBP700_ADJUSTED',
 'BBP700_ADJUSTED_ERROR',
 'BBP700_ADJUSTED_QC',
 'BBP700_QC',
 'BBP700_dPRES',
 'CDOM',
 'CDOM_ADJUSTED',
 'CDOM_ADJUSTED_ERROR',
 'CDOM_ADJUSTED_QC',
 'CDOM_QC',
 'CDOM_dPRES',
 'CHLA',
 'CHLA_ADJUSTED',
 'CHLA_ADJUSTED_ERROR',
 'CHLA_ADJUSTED_QC',
 'CHLA_QC',
 'CHLA_dPRES',
 'CYCLE_NUMBER',
 'DOXY',
 'DOXY_ADJUSTED',
 'DOXY_ADJUSTED_ERROR',
 'DOXY_ADJUSTED_QC',
 'DOXY_QC',
 'DOXY_dPRES',
 'JULD',
 'LATITUDE',
 'LONGITUDE',
 'NITRATE',
 'NITRATE_ADJUSTED',
 'NITRATE_ADJUSTED_ERROR',
 'NITRATE_ADJUSTED_QC',
 'NITRATE_QC',
 'NITRATE_dPRES',
 'PH_IN_SITU_TOTAL',
 'PH_IN_SITU_TOTAL_ADJUSTED',
 'PH_IN_SITU_TOTAL_ADJUSTED_ERROR',
 'PH_IN_SITU_TOTAL_ADJUSTED_QC',
 'PH_IN_SITU_TOTAL_QC',
 'PH_IN_SITU_TOTAL_dPRES',
 'PLATFORM_NUMBER',
 'PRES',
 'PRES_ADJUSTED',
 'PRES_ADJUSTED_ERROR',
 'PRES_ADJUSTED_QC',
 'PRES_QC',
 'PSAL',
 'PSAL_ADJUSTED',
 'PSAL_ADJUSTED_ERROR',
 'PSAL_ADJUSTED_QC',
 'PSAL_QC',
 'PSAL_dPRES',
 'TEMP',
 'TEMP_ADJUSTED',
 'TEMP_ADJUSTED_ERROR

In [ ]:
df["PSAL_ADJUSTED"]

In [ ]:
# Example with metadata file
sel = [("PLATFORM_NUMBER", "==", 6990526)]
df = pd.read_parquet( parquet_filename , engine='pyarrow', filters = sel )
df[ ["DATA_TYPE", "DATE_CREATION"] ]

#### Testing conversion

The following cell performs integrity tests on the parquet files. As the number of floats, profiles, and variables is large, the check is performed over all the variables, but not all the files. For each variable in `VARS`, files are randomly selected from the input list (in a number set to 5% of the .nc files) and for each of them, the selected `VARS` is compared to the one obtained from the parquet file. Each of these checks can:
* succeed, if the nc and parquet variables are equal
* fail, if the nc and parquet variables are not equal
* be skipped, if the randomly selected file does not contain the selected variable

If a file is skipped, another one is randomly selected, until a minimum number of files that contain the selected variable is found. For each variable, no file can be randomly picked two or more times (it can happen across variables).

The variables `successes` and `fails` contain the file id and the name of the variable for which the check was succesful or failed.

In [ ]:
def checkVars(rank, flist, VARS):

    rank_str = "#" + str(rank) + ": "
    
    rand_max = len(flist)
    nb_of_checks_per_var = np.min( [5,len(flist)] ) #int(np.ceil(rand_share*rand_max))
    nb_of_checks = nb_of_checks_per_var*len(VARS)

    print(rank_str + "Checking " + str(nb_of_checks) + " random files.")
    
    check_nb = 0
    successes = []
    fails = []
    skipped = []
    for v in range(len(VARS)):
    
        rand_idces = []
        target_var = VARS[v]
    
        r = 0
        while ((r < nb_of_checks_per_var) and (len(rand_idces) < len(flist) )):
            print(rank_str + "Check " + str(r) + " of " + str(nb_of_checks_per_var) )
            
            check_nb += 1
            rand_avail = np.arange(0,rand_max)[~np.isin(np.arange(0,rand_max), rand_idces)]
            rand_idx = np.random.choice( rand_avail )
            rand_idces.append(rand_idx)

            try:
                ref_ds = xr.load_dataset(flist[rand_idx], engine="argo")
            except:
                print(rank_str + 'Failed on ' + str(flist[rand_idx]))
                continue
                
            # print(rank_str + "Reading file " + flist[rand_idx] )
            ref_platform = ref_ds.PLATFORM_NUMBER.values[0]
        
            invars = list(set(VARS) & set(list(ref_ds.data_vars)))
            
            if target_var not in invars:
                ref_var = None
                del ref_var
                gc.collect()

                skipped.append( (rand_idx, target_var ) )
                # print(rank_str + "Current random file does not contain variable " + target_var + ", skipping this check.")
                continue
                
            print(rank_str + "Checking " + target_var + " in platform number " + str(ref_platform) + ".")
        
            dim0 = ref_ds.sizes["N_PROF"]
            dim1 = ref_ds.sizes["N_LEVELS"]
        
            if np.issubdtype(ref_ds[target_var].dtype, np.datetime64):
                ref_var = np.zeros( dim0*dim1, dtype='datetime64[ns]' )
            else:
                ref_var = np.zeros( dim0*dim1, dtype=np.float64 )
        
            for j in range(dim0):
                for k in range(dim1):
                    ref_id = j*dim1+k
                    if len(ref_ds[target_var].dims) > 1:
                        ref_var[ref_id] = ref_ds[target_var][j,k].values
                    else:
                        ref_var[ref_id] = ref_ds[target_var][j].values
        
            sel_pqt = [("PLATFORM_NUMBER", "==", ref_platform)]

            try:
                df_pqt = pd.read_parquet( outdir_pqt , engine='pyarrow', filters = sel_pqt )
            except:
                print("Loading parquet file failed for platform " + str(ref_platform) + "!")
                continue
    
            if target_var not in df_pqt.columns.tolist():
                fails.append( (rand_idx, target_var ) )
                r += 1
                print(rank_str + "Warning: " + target_var + " not found in parquet file.")
                continue
        
            df_pqt_var = df_pqt[target_var].values
        
            success = np.array_equal(ref_var, df_pqt_var, equal_nan=True)

            ref_var = None
            df_pqt_var = None
            del ref_var
            del df_pqt_var
            gc.collect()
            
            if success:
                successes.append( (flist[rand_idx], target_var ) )
            else:
                fails.append( (flist[rand_idx], target_var ) )
    
            r += 1

    print(rank_str + "All checks in process done")
    print(rank_str +  str(len(successes)) + " checks were succesful.")
    print(rank_str +  str(len(fails)) + " checks failed.")

    return successes, fails

############################################################################################################

nb_of_checks = len(flist)

NPROC = 20
CHUNK_SZ = int(np.ceil(nb_of_checks/NPROC))
chunks = batched(flist,CHUNK_SZ)

print(CHUNK_SZ)

# print(list(chunks))

print("Checking " + str(nb_of_checks) + " random files.")
print("")

# print([(rank, chunk) for rank, chunk in enumerate(chunks)])
pool_obj = multiprocessing.Pool(processes=NPROC)
outputs = pool_obj.starmap(checkVars, [(rank, chunk, VARS) for rank, chunk in enumerate(chunks)])
pool_obj.close()
print("")
print("All checks were done.")

successes = []
fails = []
for output in outputs:
    successes.append(output[0])
    fails.append(output[1])

print("Successful tests file names and variable name.")
print(successes)
print("Failed tests file names and variable name.")
print(fails)

#### Testing conversion

The following cell checks that the parquet files contain all the floats, by checking that all original platform numbers exist. Note: it *does not* check that the variables of the original float exist and are correct in the parquet file (see previous cell for this).

In [ ]:
def checkPlatformNb(rank,flist):

    rank_str = "#" + str(rank) + ": "
    outdir_pqt = '/vortexfs1/share/boom/data/nc2pqt_test/PQT/'
    
    check_nb = 0
    successes = []
    fails = []
    failed_on_read = []

    for idx in range(len(flist)):
    
        check_nb += 1
        try:
            ref_ds = xr.load_dataset(flist[idx], engine="argo")
        except:
            print(rank_str + 'Failed on ' + str(flist[idx]))
            failed_on_read.append(flist[idx])
            continue
        
        ref_platform = ref_ds.PLATFORM_NUMBER.values[0]

        ref_ds = None
        del ref_ds
        gc.collect()
        
        sel_pqt = [("PLATFORM_NUMBER", "==", ref_platform)]
        try:
            df_pqt = pd.read_parquet( outdir_pqt , engine='pyarrow', filters = sel_pqt )
        except:
            fails.append( (idx ) )
            continue

        df_pqt = None
        del df_pqt
        gc.collect()
        successes.append( (idx ) )
        
        if check_nb%10:
        print(rank_str + "Check " + str(check_nb) + " of " + str(len(flist)) + " done.")

    print(rank_str + "All checks in process done")
    print(rank_str +  str(len(successes)) + " checks were succesful.")
    print(rank_str +  str(len(fails)) + " checks failed.")
    if len(failed_on_read)>0:
        print(rank_str +  str(len(failed_on_read)) + " original Argo file(s) could not be loaded, likely due to errors in the original file. These files were likely never converted to parquet.")
        print("File list:")
        print(failed_on_read)
    else:
        print(rank_str +  str(len(failed_on_read)) + " original Argo file(s) could not be loaded.")

############################################################################################################

flist = glob.glob("/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/coriolis/*/*_Sprof.nc")

nb_of_checks = len(flist)

NPROC = 20
CHUNK_SZ = int(np.ceil(nb_of_checks/NPROC))
chunks = batched(flist,CHUNK_SZ)

print(CHUNK_SZ)

# print(list(chunks))

print("Checking " + str(nb_of_checks) + " random files.")
print("")

pool_obj = multiprocessing.Pool(processes=NPROC)
pool_obj.starmap(checkPlatformNb, [(rank, chunk) for rank, chunk in enumerate(chunks)])
pool_obj.close()

print("")
print("All checks were done.")

### Example loading Sprof from snapshot
```
ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc')
df = ds.to_dataframe()
```

In [ ]:

from dotenv import load_dotenv
load_dotenv()
import os
from pyarrow import fs
s3 = fs.S3FileSystem(region='us-east-1')


In [ ]:
s3

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import Table

ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc',engine="argo")
df = ds[['DOXY','PRES','NITRATE','PLATFORM_NUMBER']].to_dataframe()

s3_filepath = 'data.parquet'

pq.write_to_dataset(
    Table.from_pandas(df),
    s3_filepath,
    filesystem=s3,
    use_dictionary=True,
    compression="snappy",
    version="2.4",
)

